<div style="background:#FFFFEE; color:#440404; padding:8px; border-radius: 4px; text-align: center; font-weight: 500;">IFQ619 - Data Analytics for Strategic Decision Makers (2024)</div>

# IFQ619 :: UA2 - Extending Analytics (40%)

**IMPORTANT:** Refer to the instructions in Canvas [UA2 - Assignment 2 - extending analytics] *BEFORE* working on this assignment.

#### REQUIREMENTS ####

1. Complete and run the code cell below to display your name, student number, and assignment option
2. Identify an appropriate question (or questions) to be addressed by your overall data analytics narrative
3. Extend your analysis in assignment 1 with:
    - the analysis of additional unstructured data using the Guardian API (See accessing the Guardian API notebook),
    - the use of one machine learning technique (as used in the class materials), and
    - identification of ethical considerations relevant to the analysis (by drawing on class materials).
4. Ensure that you include documentation of your thinking and decision-making using markdown cells
5. Ensure that you include appropriate visualisations, and that they support the overall narrative
6. Ensure that your insights answer your question/s and are appropriate to your narrative. 
7. Ensure that your insights are consistent with the ethical considerations identified.

**NOTE:** you should not repeat the analysis from assignment 1, but you may need to save dataframes from assignment 1 and reload for use in this assignment. You may also summarise your assignment 1 insights as part of the process of identifying questions for analysis.

#### SUBMISSION ####

1. Create an assignment 2 folder named in the form **UA2-surname-idnumber** and put your notebook and any data files inside this folder. Note, do not put large training data in this folder (reference any training data that you used but keep it outside this folder), only keep small data files and models in this folder with your notebook.
2. When you have everything in the correct folder, reset all cells and restart the kernel, then run the notebook completely, checking that all cells have run without error. If you encounter errors, fix your notebook and re-run the process. It is important that your notebook runs without errors only requiring the files in the folder that you have created.
3. When the notebook is error free, zip the entire folder (you can select download folder in Jupyter).
4. Submit the zipped folder in Canvas


In [21]:
# Complete the following cell with your details and run to produce your personalised header for this assignment

from IPython.display import HTML

# personal details
first_name = "Morgan"
last_name = "Meeuwissen"
student_number = "n12240800"

personal_header = f"<h1>{first_name} {last_name} ({student_number})</h1>"
HTML(personal_header)

---


# Themes that use topic modelling?
* Long term affects of funding? still remain in news or short-lived?
* Funding as a result of news (public image leading funding)
* Funding to be lagged one year from the news? Gov funding cyle?
* Impact on Local Jobs: The Guardian’s coverage of labor market issues and economic policies provides a context for evaluating how programs like Advance Queensland impact local employment. Discussions on job creation and displacement relate directly to the ethical consideration of benefiting local communities through such initiatives.

Word limit: 1500–3000 words. Three-four analytics cycles?

**Outline:**

## Context 
from first assignment and final insights + graph? - fairly distributed -> Overarching goals of Advance Queensland... Public perception and does it make a difference?

Ethics: Bias?

https://advance.qld.gov.au/__data/assets/pdf_file/0008/1875878/Deadly-Innovation-Strategy.pdf

https://advance.qld.gov.au/innovation-in-queensland/innovation-stories/deadly-partnership-first-nations-business

# Question 1
What does the media care about show?  How much of the media attention is for indigenous matters? 

### Data
Data source is biased - Avoid further bias by not over-onstraining intial data


### Analysis

Explain Cleaning of data for TDIDF - lemmy, preproccessing, stop words, bigrams

Explain choice of NMf


### Visualisation

Clusters and Weightings


### Insight
Regional not som important

Visualisation Topics Cumulative clusters bar chart by articles.


Early detection of chronic health conditions using AI prediction model- Xujuan Zhou 360k

# Question 2 


### Data
Data: count articles on the main topic and re-introduce Advance QUeensland data with manual extra flags as 1/0

### Analysis
Make a Dataframe and show stats per column?


### Visualisation
Correlelagram


### Insight
Media attention not valid?



ANalysis:



# Question 3
What impacts funding ?  What impacts media attention from the funding 


### Data


### Analysis


### Visualisation
Regression Test confusion matrix

### Insight

Can a relationship be established? Predict funding from public opinion.


Insight: It doesn't work for a linear regression?





# Closing statement


CAN WE TOPIC MODEL ACROSS MULTIPLE CORPUS? TOPICS FROM THE PROGRAM NAME/PROJECT NAME AND FIND MATCHES IN THE GUARDIAN
https://towardsdatascience.com/unsupervised-nlp-topic-models-as-a-supervised-learning-input-cf8ee9e5cf28
rain LDA Model on 100,000 Restaurant Reviews from 2016
Grab Topic distributions for every review using the LDA Model
Use Topic Distributions directly as feature vectors in supervised classification models (Logistic Regression, SVC, etc) and get F1-score.
Use the same 2016 LDA model to get topic distributions from 2017 (the LDA model did not see this data!)
Want a topic that combines regional and indigenous? bigram traditional owner

# Past assignment Andrew notes
d your assignment 1 analysis by including consideration of ethical and human factors, including 1 extended analysis technique, and using at least 1 additional data source. Possible additional data sources:

    Guardian API (See accessing the Guardian API notebook)
    Hospital location data (Provided, see data folder)

**NOTE: you should not repeat the analysis from assignment 1, but you may need to save dataframes from assignment 1 and reload for use in this assignment. You may also summarise your assignment 1 insights as part of the process of identifying questions for analysis.**

# Insights
Start off by working out what the QLD gov articles actually show. This will be done through topic modelling variants and then clustering
articles json only has the title as a key and the text, but the title has the date that can be extracted for potential use.


clustering to topics already done lda and nmf?  TFIDF is doc specific, others take into account the whole corpus
Establish topics that match Advance Queensland and then use as predictors by year?

No mentions of Advance QLD
QLD Gov search not finding a cluster for Regional
**Try working from regional queensland back? Just regional? If taking a specific approach like this could have way less clusters....**

https://open-platform.theguardian.com/explore/ very useful. Narrow down from 79

In [22]:
# Import the necessary libraries
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, ENGLISH_STOP_WORDS
from sklearn.decomposition import LatentDirichletAllocation, NMF

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report



import pandas as pd
import json
import numpy as np
import re
import spacy
import plotly.express as px

In [23]:
# # Load the data - articles from The Guardian about the war in Ukraine
# file_path = "data/"
# file_name = "qld_articles.json"

# with open(f"{file_path}{file_name}",'r', encoding='utf-8') as fp:
#     articles = json.load(fp)

# #print(f"Loaded {len(articles)} articles from {file_name}")


# # AYOOOOOO WE DOING A LEMMATISING FUNCTION
# # https://jonathansoma.com/lede/image-and-sound/text-analysis/text-analysis-word-counting-lemmatizing-and-tf-idf/


# nlp = spacy.load("en_core_web_sm")
# # Source: spaCy is a dream, but a dream where sometimes your legs won’t move right and you can’t read text. But sometimes you can fly! So yes, as always, ups and downs
# # Morgan: I hate this
# # python -m spacy download en
# # python -m pip install spacy

# def lemmatize(text):
#     doc = nlp(text)
#     # Turn it into tokens, ignoring the punctuation
#     tokens = [token for token in doc if not token.is_punct]
#     # Convert those tokens into lemmas, EXCEPT the pronouns, we'll keep those.
#     lemmas = [token.lemma_ if token.pos_ != 'PRON' else token.orth_ for token in tokens]
#     return lemmas


# # Only count terms that in maximum of 75% of documents, and a minimum of 2 documents. 
# # Count a maximum of 10000 terms, and remove common english stop words
# StopWords = list(ENGLISH_STOP_WORDS.union(["Monday","Tuesday","Wednesday","Thursday","Friday", "Saturday", "Sunday","nbsp", "\n", "|", "\n ", 
#                                            "$", "year", "m", "new", "need", "increase","bst", "gmt", "says", "year", "told","day","know", "We", 
#                                            "'ve","'re", "read", "today", "day","like","I", "'ve","I've","I'm","just","use", "think", "story"]))


In [24]:
# # TFIDF
# # Only count terms that in maximum of 75% of documents, and a minimum of 5 documents. 
# # Count a maximum of 10000 terms, and remove common english stop words
# tfidf_vectorizer = TfidfVectorizer(
#                                    #reprocessor=lambda x: re.sub(r"([\d\.])+", "NUM", x),
#                                    max_df=0.75,min_df=5,max_features=10000,
#                                    stop_words=StopWords, #Add stop words
#                                    #tokenizer=lemmatize, # Lemmatiseer
#                                    ngram_range = (1,2) #Use Bigrams as well to pick up things like "First Nation"
# )


# tfidf_dt_matrix = tfidf_vectorizer.fit_transform(articles.values())

Introduce NMF

In [25]:
# # Set number of topics
# num_topics = 20
# # Set max number of iteractions
# max_iterations = 2500
# feature_names = tfidf_vectorizer.get_feature_names_out()

# # Create the model
# nmf_model = NMF(n_components=num_topics,init='random', random_state=37  # Set random state to have reproducible results
#                 ,beta_loss='frobenius', max_iter=max_iterations)

# # Fit the model to the data and use it to transform the data
# doc_topic_nmf = nmf_model.fit_transform(tfidf_dt_matrix)

# topic_term_nmf = nmf_model.components_

In [26]:
# # Get the topics and their terms
# nmf_topic_dict = {}
# for index, topic in enumerate(topic_term_nmf):
#     zipped = zip(feature_names, topic)
#     top_terms=dict(sorted(zipped, key = lambda t: t[1], reverse=True)[:10])
#     #print(top_terms)
#     top_terms_list= {key : round(top_terms[key], 4) for key in top_terms.keys()}
#     nmf_topic_dict[f"nmf{index}"] = top_terms_list

# nmf_terms_df = []
# # Print the topics with their terms    
# for k,v in nmf_topic_dict.items():
#     #print(k)
#     #print(v)
#     #print()
#     for t, w in v.items():
#         nmf_terms_df.append([k, t, w])

# nmf_terms_df = pd.DataFrame(nmf_terms_df)
# nmf_terms_df.rename(columns={nmf_terms_df.columns[0]: "Topic Cluster",
#                                        nmf_terms_df.columns[1]: "Term",
#                                        nmf_terms_df.columns[2]: "Weight" }, inplace=True)

# topic_map_dict = {"nmf0":"Politics 1", 
#                   "nmf1":"Youth Crime",
#                   "nmf2":"Family",
#                   "nmf3":"Politics 2",
#                   "nmf4":"Environment",
#                   "nmf5":"COVID 1",
#                   "nmf6":"Aged Care",
#                   "nmf7":"Mining",
#                   "nmf8":"Power",
#                   "nmf9":"Police",
#                   "nmf10":"Great Barrier Reef",
#                   "nmf11":"Flood",
#                   "nmf12":"Education",
#                   "nmf13":"Domestic Violence",
#                   "nmf14":"Sport",
#                   "nmf15":"Cost of Living",
#                   "nmf16":"Other",
#                   "nmf17":"Climate Change",
#                   "nmf18":"Indigenous",
#                   "nmf19":"COVID 2"}

# # Write the Terms DF to a csv for later use
# nmf_terms_df = nmf_terms_df.replace({"Topic Cluster": topic_map_dict})
# nmf_terms_df.to_csv("data/nmf_terms_df.csv")

# # Write the topics DF to a csv for later use
# doc_topic_nmf_df =  pd.DataFrame(doc_topic_nmf,index=articles.keys(), columns= nmf_model.get_feature_names_out())
# doc_topic_nmf_df = doc_topic_nmf_df.rename(columns = topic_map_dict)
# doc_topic_nmf_df.reset_index(inplace= True)
# doc_topic_nmf_df.to_csv("data/nmf_topics_df.csv", index = False)


In [27]:
nmf_terms_df = pd.read_csv("data/nmf_terms_df.csv")


In [28]:
# Your answer here




nmf_topic_terms_fig = px.bar(nmf_terms_df,
       x ="Weight",
       y = "Term",
       facet_col="Topic Cluster", 
       facet_col_wrap=5,
       orientation='h',
       title = "Total Contractual Commitment ($ GST excl.) by Local Government Area",
       labels = {"Local Government Area (LGA)": "Local Government Area", 
                 "Total Contractual Commitment ($ GST excl.)": "Contractual Commitment ($ GST excl.)"})

nmf_topic_terms_fig.update_layout(
    title_font_size=25,
    title_x=0.5,
    legend_title_font_size=15,
    height=1000
)
nmf_topic_terms_fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1])) # https://plotly.com/python/facet-plots/
nmf_topic_terms_fig.update_yaxes(showticklabels=True, matches=None)



In [29]:
nmf_topics_df = pd.read_csv("data/nmf_topics_df.csv")

# Below gets the topics 
nmf_topics_df = nmf_topics_df.rename(columns={"index":"article"})

nmf_topics_df["article_year"] = nmf_topics_df["article"].str.extract(r"(?<=\[)(\d{4})(?=\-)").astype(int)

nmf_topics_df = nmf_topics_df.drop('article', axis=1)



#Normalise the weights into proportional topic representation within each article
nmf_topics_df.set_index("article_year", inplace= True)
nmf_topics_df = nmf_topics_df.divide(nmf_topics_df.sum(axis =1), axis = 0).reset_index()


nmf_annual_topic_df = nmf_topics_df.melt(id_vars=["article_year"], var_name="Topic", value_name = "aggregated_mentions")
nmf_annual_topic_df = nmf_annual_topic_df.groupby(["article_year","Topic"]).sum().reset_index()

In [30]:
px.bar(nmf_annual_topic_df, 
       x = "Topic", 
       y = "aggregated_mentions", 
       color = "article_year")

Ingest Advance Queensland Data

In [31]:
advance_df = pd.read_csv("data/cleaned_advance_queensland.csv")


combined_df = pd.merge(advance_df, , left_on="Approval Year", right_on = "article_year")
combined_df = combined_df[combined_df["Indigenous_Related"] == 1].reset_index()

combined_df["Is_Regional"] = combined_df["LGA Status"] == "Regional Councils"

combined_df = combined_df[["aggregated_mentions", "Is_Regional", "Medical_Related" , "Approval Year", "Contractual Commitment ($ GST excl.)"]]
# May need to split this out again if want to do a regression

In [32]:
advance_corr = combined_df.corr()

px.imshow(advance_corr)

In [33]:
px.scatter_matrix(combined_df)

In [34]:
# Break the current dataset into train and test datasets
x_train, x_test, y_train, y_test = train_test_split(combined_df[["aggregated_mentions", "Is_Regional", "Medical_Related"]], 
                                                    combined_df[["Contractual Commitment ($ GST excl.)"]],
                                                    shuffle=True, train_size=0.8, random_state=99) # Train size determines the perceptage use for training the model

In [35]:
# Create a new linear regression model
linear_model = LinearRegression() 

# Train the model with the train dataset
linear_model.fit(x_train, y_train) 

# Predict using the testing dataset
linear_predictions = linear_model.predict(x_test) 
linear_predictions

array([[ 54729.94242346],
       [ 54617.77265842],
       [290862.36674522],
       [133609.28260246]])

In [36]:
# Calculate the error between the testing dataset the model's predictions
linear_R2 = r2_score(y_test, linear_predictions) 

print(f'The model R squared score is: {linear_R2}')


The model R squared score is: -3.200862210544953


In [37]:
fund_fig_df=pd.DataFrame(y_test)
fund_fig_df["Prediction"]= linear_predictions
# This is so much simpler than the lecture materials...


#linreg_fig.update_layout(hovermode="x unified")


linreg_fig = px.scatter(fund_fig_df)
linreg_fig.show()

In [38]:
fund_fig_df

,Contractual Commitment ($ GST excl.),Prediction
0,43750,54729.942423
11,100000,54617.772658
7,150000,290862.366745
6,72986,133609.282602


Potential logistic regression on idenifying medical

In [52]:

combined_df = pd.merge(advance_df, nmf_annual_topic_df[nmf_annual_topic_df["Topic"] == "COVID 1"], left_on="Approval Year", right_on = "article_year")
combined_df = combined_df.reset_index()

combined_df["Is_Regional"] = combined_df["LGA Status"] == "Regional Councils"

combined_df = combined_df[["aggregated_mentions", "Is_Regional", "Medical_Related" , "Approval Year", "Contractual Commitment ($ GST excl.)"]]

#This is obviously bad at predicting the patients with actual thyroid cancer... need to focus only on the related variables
px.imshow(combined_df.corr())

In [40]:
# Break the current dataset into train and test datasets
x_train, x_test, y_train, y_test = train_test_split(combined_df[["aggregated_mentions", "Is_Regional", "Contractual Commitment ($ GST excl.)"]], 
                                                    combined_df[["Medical_Related"]], 
                                                    shuffle=True, train_size=0.8, random_state=99) # Train size determines the perceptage use for training the model


# Scale the x data
scale = StandardScaler()
x_train = scale.fit_transform(x_train)
x_test = scale.transform(x_test)

In [41]:
# Check the class balance
y_train.value_counts(normalize=True)

Medical_Related
0                  0.873016
1                  0.126984
Name: proportion, dtype: float64

In [43]:
# Set up the logistic model object with weightings
logistic_model = LogisticRegression(class_weight={0: 0.126984, 1:  0.873016})

# Fit the logistic model
logistic_model.fit(x_train, y_train)

/home/mmeeuwissen/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



LogisticRegression(class_weight={0: 0.126984, 1: 0.873016})

In [49]:
type(combined_df[["aggregated_mentions", "Is_Regional" , "Approval Year", "Contractual Commitment ($ GST excl.)"]])

pandas.core.frame.DataFrame

In [51]:
logistic_prediction = logistic_model.predict(x_test) # Use the model to predict based on the testing dataset

cm = confusion_matrix(y_test, logistic_prediction) # Compare the model's prediction against the true value in the testing dataset

cm_fig = px.imshow(cm, labels={'x': 'Predicted label', 'y': 'Actual label'})
cm_fig.show()

report = classification_report(y_test, logistic_prediction) # Get further evaluation metrics
print(report)

              precision    recall  f1-score   support

           0       0.94      0.73      0.82        70
           1       0.24      0.67      0.35         9

    accuracy                           0.72        79
   macro avg       0.59      0.70      0.59        79
weighted avg       0.86      0.72      0.77        79



### **References**

Queensland Government Open Data Portal. (2024, July 1st) _Advance Queensland Funding Recipients_.

    Retrieved August 12, 2024, from https://www.data.qld.gov.au/dataset/advance-queensland-funding-recipients